In [ ]:
import json
import pandas as pd
import time
import random
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def get_sample_place_data(restaurant_name):
    """음식점 정보 크롤링"""
    # 검색 URL 생성
    search_url = f"https://map.naver.com/p/search/{restaurant_name} 강남구"
    
    # Selenium 설정
    options = webdriver.ChromeOptions()
    # options.add_argument('--headless=new')  # 헤드리스 모드
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(search_url)
    
    try:
        wait = WebDriverWait(driver, 3)

        # entryIframe(상세 정보)가 바로 떠 있는지 먼저 확인
        try:
            wait.until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#entryIframe"))
            )
            detail_iframe_found = True
        except:
            detail_iframe_found = False

        if not detail_iframe_found:
            # 검색 결과 iframe 진입
            wait.until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#searchIframe"))
            )
            driver.switch_to.frame("searchIframe")
            
            # restaurant_name이 들어간 span을 가진 a 태그 클릭
            target_xpath = f"//a[@role='button'][.//span[contains(text(), '{restaurant_name}')]]"

            first_result = driver.find_element(By.XPATH, target_xpath)
            first_result.click()
            
            driver.switch_to.default_content()
            # 상세 정보 iframe이 뜰 때까지 대기
            wait.until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#entryIframe"))
            )

        driver.switch_to.frame("entryIframe")

        # 데이터 추출
        data = {}

        # 1. 음식점 기본 정보

        # 음식점 명
        try:
            # name_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "span.GHAhO")))
            name_element = driver.find_element(By.CSS_SELECTOR, "span.GHAhO")
            data['음식점_이름'] = name_element.text
        except Exception as e:
            data['음식점_이름'] = "정보 없음"

        # 음식점 사진
        try:
            place_img_elem = driver.find_element(By.CSS_SELECTOR, "div.fNygA img")
            place_img_url = place_img_elem.get_attribute("src")
            data['음식점_사진'] = place_img_url
        except Exception:
            data['음식점_사진'] = "정보 없음"

        # 주소
        try:
            address_element = driver.find_element(By.CSS_SELECTOR, "span.LDgIH")
            data['주소'] = address_element.text
        except Exception as e:
            data['주소'] = "정보 없음"

        # 카테고리
        try:
            category_element = driver.find_element(By.CSS_SELECTOR, "span.lnJFt")
            data['카테고리'] = category_element.text
        except Exception as e:
            data['카테고리'] = "정보 없음"

        # 영업시간
        try:
            # 영업시간 펼치기(필요시)
            openhour_elems = driver.find_elements(By.CSS_SELECTOR, "div.H3ua4")
            if not openhour_elems:
                openhour_btn = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a.gKP9i.RMgN0")))
                openhour_btn.click()

            # 모든 요일별 영업시간 추출
            openhour_rows = driver.find_elements(By.CSS_SELECTOR, "span.A_cdD")
            openhours = []
            for row in openhour_rows:
                try:
                    day = row.find_element(By.CSS_SELECTOR, "span.i8cJw").text
                    hours_raw = row.find_element(By.CSS_SELECTOR, "div.H3ua4").text
                    hours = hours_raw.replace('\n', '; ')  # 줄바꿈을 세미콜론+공백으로 변환
                    openhours.append(f"{day}: {hours}")
                except Exception as e:
                    continue

            # 리스트 형태로 저장하거나, 문자열로 합칠 수 있음
            data['영업시간'] = "; ".join(openhours) if openhours else "정보 없음"
        except Exception as e:
            data['영업시간'] = "정보 없음"

        # 전화번호
        try:
            phone_element = driver.find_element(By.CSS_SELECTOR, "span.xlx7Q")
            data['전화번호'] = phone_element.text
        except Exception as e:
            data['전화번호'] = "정보 없음"

        # 2. 리뷰 데이터
        try:
            # 리뷰 탭 클릭
            review_tab = wait.until(EC.element_to_be_clickable((By.XPATH, "//span[normalize-space(text())='리뷰']")))
            review_tab.click()
            time.sleep(0.5)
            # 1. 모든 태그가 보일 때까지 더보기(화살표) 버튼 반복 클릭
            while True:
                try:
                    more_btn = wait.until(
                        EC.element_to_be_clickable((By.CSS_SELECTOR, "a.dP0sq"))
                    )
                    driver.execute_script("arguments[0].click();", more_btn)
                except Exception:
                    break  # 더이상 버튼이 없으면 종료

            # 2. 모든 태그 리뷰 추출
            tag_elements = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "li.MHaAm")))
            tags = {}
            if tag_elements:
                for element in tag_elements:
                    try:
                        # 태그 텍스트
                        tag_text = element.find_element(By.CSS_SELECTOR, "span.t3JSf").text.strip('"')
                        # 인원수 (숫자만 추출)
                        count_text = element.find_element(By.CSS_SELECTOR, "span.CUoLy").text
                        count = ''.join(filter(str.isdigit, count_text))
                        tags[tag_text] = int(count) if count else 0
                    except Exception:
                        continue
            # 없으면 빈 딕셔너리
            data['음식점_태그'] = json.dumps(tags, ensure_ascii=False)

            # 리뷰 추출
            reviews = []
            review_lis = driver.find_elements(By.CSS_SELECTOR, "li.place_apply_pui.EjjAW")[:2]
            for li in review_lis:
                # 태그 추출
                try:
                    tag = li.find_element(By.CSS_SELECTOR, "div.pui__HLNvmI span.pui__jhpEyP").text.strip()
                except Exception:
                    tag = None
                # 게시글 추출 (줄바꿈 처리)
                try:
                    post_elem = li.find_element(By.CSS_SELECTOR, "div.pui__vn15t2 a[data-pui-click-code='rvshowmore']")
                    post = post_elem.text.strip()
                except Exception:
                    post = None
                # 3. 이미지 URL 추출 (여러 장 가능)
                review_img_urls = []
                try:
                    driver.execute_script("arguments[0].scrollIntoView();", li)
                    time.sleep(0.3)

                    imgs = li.find_elements(By.CSS_SELECTOR, "div.HH5sZ img")
                    for img in imgs[:3]:
                        src = img.get_attribute("src") or img.get_attribute("data-src") or img.get_attribute("srcset")
                        if src:
                            review_img_urls.append(src)
                except Exception:
                    review_img_urls = []


                reviews.append({
                    "태그": tag,
                    "게시글": post,
                    "이미지": review_img_urls
                })
            data['리뷰'] = json.dumps(reviews, ensure_ascii=False)


        except Exception as e:
            data['리뷰'] = {}
        # 3. 메뉴 정보
        try:
            menu_tab = wait.until(EC.element_to_be_clickable((By.XPATH, "//span[contains(text(), '메뉴')]")))
            menu_tab.click()
            time.sleep(0.3)

            # 공통 스크롤 처리
            scroll_pause_time = 0.1
            last_height = driver.execute_script("return document.body.scrollHeight")
            for i in range(0, last_height, 1080):
                driver.execute_script(f"window.scrollTo(0, {i});")
                time.sleep(scroll_pause_time)
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(scroll_pause_time)

            menus = []

            # 1. order_list_item
            menu_items = driver.find_elements(By.CSS_SELECTOR, "li.order_list_item")
            for item in menu_items:
                try:
                    name = item.find_element(By.CSS_SELECTOR, "div.tit").text.strip()
                    price = item.find_element(By.CSS_SELECTOR, "div.price > strong").text.strip()
                    if name and price:
                        menus.append({"메뉴명": name, "가격": price})
                except:
                    continue

            # 2. E2jtL 구조
            menu_items = driver.find_elements(By.CSS_SELECTOR, "li.E2jtL")
            for item in menu_items:
                try:
                    name = item.find_element(By.CSS_SELECTOR, "span.lPzHi").text.strip()
                    price = item.find_element(By.CSS_SELECTOR, "div.GXS1X em").text.strip()
                    if name and price:
                        menus.append({"메뉴명": name, "가격": price})
                except:
                    continue

            # 중복 제거
            unique = set()
            deduped_menus = []
            for menu in menus:
                key = (menu['메뉴명'], menu['가격'])
                if key not in unique:
                    unique.add(key)
                    deduped_menus.append(menu)

            data['메뉴_정보'] = json.dumps(deduped_menus, ensure_ascii=False)
        
        except Exception as e:
            data['메뉴_정보'] = []
        
        return data
    except Exception as e:
        return None
    
    finally:
        driver.quit()


In [ ]:
def collect_all_restaurant_data(restaurants_df, start_idx):
    for i in range(start_idx, len(restaurants_df)):
        name = restaurants_df.loc[i, '음식점명']
        print('*' * 50)
        print(f'{i}번째 음식점: {name} 수집 시작')

        try:
            data = get_sample_place_data(name)

            if data:
                df_one = pd.DataFrame([data])
                # 파일 경로 설정
                file_path = '../../data/external/sample_place_restaurant_data.csv'
                # 파일 존재 여부 확인
                header = not os.path.exists(file_path)
                # DataFrame을 CSV로 저장
                df_one.to_csv(file_path, mode='a', header=header, index=False, encoding='utf-8-sig')
                print(f'{i}번째 음식점: {name} 저장 완료')
            else:
                print(f'{i}번째 음식점: {name} 데이터 없음')

        except Exception as e:
            print(f'오류 발생: {e} → {i}번째 음식점: {name} 수집 실패')

        time.sleep(random.uniform(3, 5))


In [ ]:
# CSV 파일에서 음식점 목록 로드
restaurants_df = pd.read_csv('../../data/interim/gangnam_restaurants_cleaned.csv')

# 전체 데이터 수집 (시간이 오래 걸릴 수 있음)
# 시작 인덱스 (gangnam_restaurants_cleaned.csv에서 -2)
i = 0
restaurants = restaurants_df.iloc[i:]
result = collect_all_restaurant_data(restaurants, i)


**************************************************
0번째 음식점: 으뜸돈까스 수집 시작
0번째 음식점: 으뜸돈까스 저장 완료
**************************************************
1번째 음식점: 캠핑고양이 수집 시작
1번째 음식점: 캠핑고양이 저장 완료
**************************************************
2번째 음식점: 웨스키하우스 수집 시작
2번째 음식점: 웨스키하우스 저장 완료
**************************************************
3번째 음식점: 육회한집샤브 수집 시작
3번째 음식점: 육회한집샤브 저장 완료
**************************************************
4번째 음식점: 연하동 신논현점 수집 시작
4번째 음식점: 연하동 신논현점 저장 완료
**************************************************
5번째 음식점: 세븐7(SEVEN7) 수집 시작
5번째 음식점: 세븐7(SEVEN7) 데이터 없음
**************************************************
6번째 음식점: 카페코너 수집 시작
6번째 음식점: 카페코너 저장 완료
**************************************************
7번째 음식점: 시선 신사점 수집 시작
7번째 음식점: 시선 신사점 저장 완료
**************************************************
8번째 음식점: 비노솔 수집 시작
8번째 음식점: 비노솔 데이터 없음
**************************************************
9번째 음식점: 자연도소금빵in도산 수집 시작
9번째 음식점: 자연도소금빵in도산 저장 완료
****************************************